In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [4]:
train = pd.read_csv('../data/train.csv', index_col="ID")
test = pd.read_csv('../data/test.csv', index_col="ID")

TypeError: Cannot convert numpy.ndarray to numpy.ndarray

In [ ]:
train.head()

In [ ]:
train.describe()

In [ ]:
test.head()

In [ ]:
test.describe()

In [ ]:
print(len(test))
print(len(train))

In [ ]:
# Check for missing values
print(f"Number of missing values in train: {train.isnull().sum().sum()}")
# print only the columns with missing values and the number of missing values per column in train, if no missing values, no output
print(train.isnull().sum()[train.isnull().sum() > 0])
# Percentage of missing values in clouds
print(f"Percentage of missing values in clouds: {train['clouds'].isnull().sum() / len(train['clouds']) * 100}")
print(f"Number of missing values in test: {test.isnull().sum().sum()}")
print(test.isnull().sum()[test.isnull().sum() > 0])


In [ ]:
# For every missing value, we will interpolate the result from the previous and next value
train = train.interpolate()
print(f"Number of missing values in train: {train.isnull().sum().sum()}")

In [ ]:
train['measurement_time'] = pd.to_datetime(train['measurement_time'])
# plot all values of target across measurement_time
plt.plot(train['measurement_time'], train['target'])
plt.xlabel('Measurement Time')
plt.ylabel('Target')
plt.title('Target vs Measurement Time')
plt.xticks(rotation=90)
plt.show()

In [ ]:
def label_season(month):
    if month in [3, 4, 5]:
        return 'spring'
    elif month in [6, 7, 8]:
        return 'summer'
    elif month in [9, 10, 11]:
        return 'fall'
    else:
        return 'winter'
def label_time_of_day(hour):
    if hour in [6,7,8,9,10,11,12]:
        return 'morning'
    elif hour in [13,14,15,16,17]:
        return 'afternoon'
    elif hour in [18,19,20,21,22]:
        return 'evening'
    else:
        return 'night'

In [ ]:
def add_lagged_features(column: pd.Series, lags=7) -> pd.Series:
    lag = []  # Initialize an empty list to store lagged values
    for i in range(len(column)):
        # Create a lag for each value in the column
        if i + lags < len(column):
            lag.append(column[i + lags])  # Append the value shifted by 'lags'
        else:
            lag.append(None)  # Append None (or np.nan) for out-of-bound indices
    
    # Convert the list of lagged values to a pandas Series and return it
    lag = pd.Series(lag, index=column.index)
    lag = lag.interpolate()
    return lag

In [ ]:
# separate time_measurement into hour, day of the week and month column
train_separate = train.copy()
train_separate['hour'] = train_separate['measurement_time'].dt.hour
train_separate['day_of_week'] = train_separate['measurement_time'].dt.dayofweek
train_separate['month'] = train_separate['measurement_time'].dt.month
#train_separate['year'] = train_separate['measurement_time'].dt.year
train_separate = train_separate.drop(columns=['measurement_time'])
# Apply season labeling based on the 'Month' column
train_separate['season'] = train_separate['month'].apply(label_season).copy()
# One-hot encode the 'Season' column
train_separate = pd.get_dummies(train_separate, columns=['season'], drop_first=False).copy()
train_separate = train_separate.drop(columns=['month'])
train_separate['time_of_day'] = train_separate['hour'].apply(label_time_of_day).copy()
# One-hot encode the 'Season' column
train_separate = pd.get_dummies(train_separate, columns=['time_of_day'], drop_first=False).copy()
train_separate = train_separate.drop(columns=['hour'])
for column_label in ['mean_room_temperature', 'outside_temperature', 'wind_speed', 'clouds']:
    train_separate[column_label + '_lag'] = add_lagged_features(train_separate[column_label])
# print the variance of each column
#print(train_separate.describe())
print(train_separate.columns)
print(f"Number of missing values in train_separate: {train_separate.isnull().sum().sum()}")
# print only the columns with missing values and the number of missing values per column in train, if no missing values, no output
print(train_separate.isnull().sum()[train_separate.isnull().sum() > 0])

In [ ]:
# Matrix of correlation between numerical columns
# only numerical columns are considered not measurement_time
train_numerical = train_separate.copy()
matrix = train_numerical.corr()
# print highest correlations var 1 var 2 corr
print(matrix.unstack().sort_values().drop_duplicates().tail(10))

plt.matshow(matrix)
# include the column names
plt.xticks(range(train_numerical.shape[1]), train_numerical.columns, fontsize=14, rotation=90)
plt.yticks(range(train_numerical.shape[1]), train_numerical.columns, fontsize=14)
plt.colorbar()
plt.show()

According to  Pearson correlation coefficient, $r > 0.75$ is considered as highly correlated. This mainly affects our results with radiation values. The following pairs of variables are highly correlated:
```
sun_radiation_north          sun_radiation_perpendicular    0.847473
sun_radiation_perpendicular  sun_radiation_south            0.861518
```
Therefore we drop `sun_radiation_perpendicular` column.

In [ ]:
X = train_numerical.copy()
X = X.drop(columns=['target'])
y = train_numerical['target']
train_size = int(len(X) * 0.8)
X_train, X_val = X[:train_size], X[train_size:]
y_train, y_val = y[:train_size], y[train_size:]

In [ ]:
print(f"X_train: {len(X_train)}, X_val: {len(X_val)}, Y_train: {len(y_train)}, Y_val: {len(y_val)}")

In [ ]:
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from math import sqrt

# Drop 'sun_radiation_perpendicular' for Version 2
X2_train = X_train.drop(columns=['sun_radiation_perpendicular'])
X2_val = X_val.drop(columns=['sun_radiation_perpendicular'])

# Standardize the features using consistent fit and transform
scaler1 = StandardScaler()
scaler2 = StandardScaler()

# Fit scaler on training data and transform both training and validation data
X1_scaled_train = scaler1.fit_transform(X_train)
X1_scaled_val = scaler1.transform(X_val)

X2_scaled_train = scaler2.fit_transform(X2_train)
X2_scaled_val = scaler2.transform(X2_val)

# Apply PCA
pca1 = PCA()
pca2 = PCA()

# Fit PCA on training data and transform both training and validation data
X1_pca_train = pca1.fit_transform(X1_scaled_train)
X1_pca_val = pca1.transform(X1_scaled_val)

X2_pca_train = pca2.fit_transform(X2_scaled_train)
X2_pca_val = pca2.transform(X2_scaled_val)

# Train linear regression models
model1 = LinearRegression()
model1.fit(X1_pca_train, y_train)

model2 = LinearRegression()
model2.fit(X2_pca_train, y_train)

model3 = LinearRegression()
model3.fit(X_train, y_train)

# Predict on the validation set
y1_pred = model1.predict(X1_pca_val)
y2_pred = model2.predict(X2_pca_val)
y3_pred = model3.predict(X_val)

# Calculate RMSE
RMSE1 = sqrt(mean_squared_error(y_val, y1_pred)) 
RMSE2 = sqrt(mean_squared_error(y_val, y2_pred)) 
RMSE3 = sqrt(mean_squared_error(y_val, y3_pred)) 

print(f"RMSE for PCA when keeping all components: {RMSE1}")
print(f"RMSE for PCA without sun_radiation_perpendicular: {RMSE2}")
print(f"RMSE for basic: {RMSE3}")

# Check explained variance ratio
print(f"Explained variance ratio for all components (X1_pca): {pca1.explained_variance_ratio_.sum():.4f}")
print(f"Explained variance ratio without sun_radiation_perpendicular (X2_pca): {pca2.explained_variance_ratio_.sum():.4f}")

In [ ]:
print(X.columns.values)

In [ ]:
train_log = X.copy()
# apply log transformation to all numerical columns except temperature
train_log['sun_radiation_north'] = np.log1p(train_log['sun_radiation_north'])
train_log['sun_radiation_perpendicular'] = np.log1p(train_log['sun_radiation_perpendicular'])
train_log['sun_radiation_south'] = np.log1p(train_log['sun_radiation_south'])
train_log['sun_radiation_east'] = np.log1p(train_log['sun_radiation_east'])
train_log['sun_radiation_west'] = np.log1p(train_log['sun_radiation_west'])
#train_log['wind_speed'] = np.log1p(train_log['wind_speed'])
#train_log['wind_direction'] = np.log1p(train_log['wind_direction'])
#train_log['source_1_temperature'] = np.log1p(train_log['source_1_temperature'])
#train_log['source_2_temperature'] = np.log1p(train_log['source_2_temperature'])
#train_log['source_3_temperature'] = np.log1p(train_log['source_3_temperature'])
#train_log['source_4_temperature'] = np.log1p(train_log['source_4_temperature'])
#train_log['mean_room_temperature'] = np.log1p(train_log['mean_room_temperature'])
train_log['outside_temperature'] = np.log1p(train_log['outside_temperature'])
train_log['clouds'] = np.log1p(train_log['clouds'])

print(f"Number of missing values in train: {train_log.isnull().sum().sum()}")
# print only the columns with missing values and the number of missing values per column in train, if no missing values, no output
print(train_log.isnull().sum()[train_log.isnull().sum() > 0])
train_log = train_log.interpolate()
print(train_log.isnull().sum()[train_log.isnull().sum() > 0])

In [ ]:
# normalise the data
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
train_normalised = X.copy()
train_normalised[train_normalised.columns] = scaler.fit_transform(train_normalised)

In [ ]:
# normalise the data
scaler = StandardScaler()
train_normalised_std = X.copy()
train_normalised_std[train_normalised_std.columns] = scaler.fit_transform(train_normalised)

In [ ]:
# Fit a linear regression model on the log and normalised data
X1 = train_log
X2 = train_normalised
X3 = train_normalised_std
X4 = train_numerical.drop(columns=['target'])

X = [X1, X2, X3, X4]
labels = [" logged", " normalised minmax", " normalised standard", ""]

for i in range(len(X)):
    data = X[i]
    datay = y
    label : str = labels[i]
    train_size = int(len(data) * 0.8)
    X_train, X_test = data[:train_size], data[train_size:]
    y_train, y_test = datay[:train_size], datay[train_size:]
    model = LinearRegression()
    model.fit(X_train, y_train)
    # Predict the target variable on the test set
    y_pred = model.predict(X_test)
    # Calculate R-squared to see how well the target is predicted by the PCA components
    rmse = sqrt(mean_squared_error(y_test, y_pred))
    # Output the amount of variance explained by the PCA components in the target
    print("RMSE with" + label + f" numerical components: {rmse}")

In [ ]:
# Do minmax and log
N = int(len(train_log) * 0.8)
X_train, X_test = train_log[:N], train_log[N:]
y_train, y_test = y[:N], y[N:]

scaler1 = MinMaxScaler()
scaler2 = MinMaxScaler()
X_train[X_train.columns] = scaler1.fit_transform(X_train)
X_test[X_test.columns] = scaler2.fit_transform(X_test)

model = LinearRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

#calculate RMSE
RMSE1 = sqrt(mean_squared_error(y_test, y_pred)) 
print(f"RMSE for minmax and log: {RMSE1}")

In [ ]:
# Do minmax and log with PCA
N = int(len(train_log) * 0.8)
X_train, X_test = train_log[:N], train_log[N:]
y_train, y_test = y[:N], y[N:]

scaler1 = MinMaxScaler()
scaler2 = MinMaxScaler()
X_train[X_train.columns] = scaler1.fit_transform(X_train)
X_test[X_test.columns] = scaler2.fit_transform(X_test)
pca = PCA()
X_pca_train = pca.fit_transform(X_train)
X_pca_test = pca.transform(X_test)

model = LinearRegression()
model.fit(X_pca_train, y_train)
y_pred = model.predict(X_pca_test)

#calculate RMSE
RMSE1 = sqrt(mean_squared_error(y_test, y_pred)) 
print(f"RMSE for PCA with minmax and log: {RMSE1}")